In [112]:
import numpy as np
np.set_printoptions(edgeitems=10,linewidth=180)

In [189]:
# inputs = '''R 4
# U 4
# L 3
# D 1
# R 4
# D 1
# L 5
# R 2'''

# inputs = '''R 5
# U 8
# L 8
# D 3
# R 17
# D 10
# L 25
# U 20'''

axis = {'U': 0, 'D': 0, 'L': 1, 'R': 1}
sign = {'U': 1, 'D': -1, 'L': -1, 'R': 1} # np likes 0,0 in top-left, elves apparently in bottpm-left
moves = []
for line in inputs.split('\n'):
    _dir, dist = line.split()
    dist2d = [0, 0]
    dist2d[axis[_dir]] = int(dist) * sign[_dir]
    moves.append(dist2d)
moves = np.array(moves, np.int32)
moves

array([[  0,  -1],
       [ -2,   0],
       [  0,   2],
       [  0,  -1],
       [ -1,   0],
       [  0,  -1],
       [  1,   0],
       [  0,   1],
       [  0,  -2],
       [  0,   2],
       ...,
       [ -4,   0],
       [ 17,   0],
       [-15,   0],
       [  0, -17],
       [ -1,   0],
       [  0,  14],
       [  0, -18],
       [  2,   0],
       [  0,  11],
       [  5,   0]], dtype=int32)

In [199]:
def sim_1step(move2d, head, tail):
    move_dist = np.abs(move2d).sum()
    assert move_dist <= 2, move2d
    old_tail = tail
    dist = np.abs(head - old_tail)
    total_dist = dist.sum()
    max_dist   = dist.max()
    if max_dist <= 1: return (tail - tail), tail # Stay put
    # Imagine head is in the second row and moves somewhere. These are all possibilities.
    # . . .
    # . .
    # T
    if   total_dist == 3 and move_dist == 1: tail = head - move2d # move diagonally, just behind
    elif total_dist == 3 and move_dist == 2: tail = old_tail + move2d # head moved diagonally, and we follow suit
    elif total_dist == 2 and move_dist == 2: tail = (head + old_tail) // 2
    elif max_dist == 2: tail = old_tail + move2d # move in tandem
    else: assert max_dist <= 1, dist
    tail_move = (tail - old_tail)
    # print(move2d, tail_move, dist, move_dist, f'H:{head} | T: {old_tail} -> {tail}')
    return tail_move, tail


position = moves.cumsum(axis=0)
position_min = position.min(axis=0)
position -= position_min

n_links = 10
links_pos = np.zeros((n_links, 2), np.int32) - position_min

max_dims = position.max(axis=0) + 1
touched = np.zeros(max_dims, np.int32)

# viz = np.zeros_like(touched)
# for i, link in enumerate(reversed(links_pos)):
#     print(link)
#     viz[tuple(link)] = n_links - i
# print('-----')
# print('START')
# # print(move2d_1step)
# print(np.flip(viz, axis=0))
# print('-----')

for move2d in moves:
    n_times = np.abs(move2d).max()
    move2d_1step = move2d // n_times
    
    for i in range(n_times):
        # print(i)
        last_link_move = move2d_1step
        links_pos[0] = links_pos[0] + move2d_1step
        for link_idx in range(1, n_links):
            # print(link_idx + 1)
            last_link_move, links_pos[link_idx] = sim_1step(last_link_move, links_pos[link_idx - 1], links_pos[link_idx])
            # print(last_link_move)
        touched[tuple(links_pos[-1])] += 1
        # viz = np.zeros_like(touched)
        # viz[tuple(-position_min)] = -1
        # for i, link in enumerate(reversed(links_pos)):
        #     viz[tuple(link)] = n_links - i
        # print('-----')
        # print(move2d)
        # # print(move2d_1step)
        # print(np.flip(viz, axis=0))
        # print('-----')

In [200]:
(touched > 0).sum()


2376

In [188]:
inputs = '''L 1
D 2
R 2
L 1
D 1
L 1
U 1
R 1
L 2
R 2
L 2
D 1
R 2
D 1
U 2
R 2
D 1
R 1
L 2
R 1
D 1
U 2
R 2
D 1
R 2
L 1
D 1
U 1
R 1
D 2
L 1
D 1
L 1
U 1
L 2
U 1
L 1
U 1
L 1
D 2
R 2
U 1
D 2
R 1
U 1
D 1
R 1
U 2
L 2
D 2
R 1
U 2
L 2
U 1
D 1
L 1
R 2
L 2
R 1
D 2
L 1
D 2
L 1
R 2
U 2
D 2
U 1
R 2
D 2
L 2
U 1
D 2
R 1
L 2
R 1
L 2
U 2
D 2
U 2
D 2
R 1
U 2
L 2
D 1
U 2
L 1
D 1
R 2
U 1
L 1
D 1
U 2
D 2
R 2
U 1
L 2
D 2
L 1
D 2
L 1
U 1
R 2
L 2
D 1
R 2
U 2
R 2
L 2
D 1
R 1
L 1
D 2
U 2
L 1
D 1
R 1
L 3
U 2
R 2
D 3
L 1
R 3
L 1
R 2
D 1
U 3
L 1
U 2
L 1
D 3
R 3
D 1
U 3
R 2
D 2
R 2
L 1
U 1
R 1
U 2
R 3
U 1
D 1
U 1
R 3
D 2
R 2
U 3
D 1
R 3
L 3
U 3
D 3
R 1
D 3
L 1
D 3
L 1
R 1
U 1
L 2
D 3
U 3
D 2
U 3
D 2
U 2
D 1
R 2
U 3
L 1
R 2
L 3
U 3
D 3
L 2
R 2
U 3
R 1
L 1
U 1
D 1
R 2
L 3
U 3
D 1
L 1
U 1
R 1
D 2
L 2
D 2
R 3
D 2
R 3
D 2
R 1
D 1
R 3
D 1
U 1
L 1
R 2
D 3
L 3
R 1
L 2
R 1
U 3
D 2
R 1
U 1
L 2
R 2
L 2
D 1
U 3
R 3
D 2
R 1
U 2
L 3
R 1
D 1
U 2
L 4
D 2
R 1
U 4
R 4
D 3
R 3
U 2
L 2
U 1
D 3
U 2
D 2
R 4
D 3
R 3
D 4
R 2
U 2
L 1
D 1
L 2
U 4
D 4
R 3
U 2
R 3
U 4
D 2
R 3
D 4
L 3
D 1
R 4
D 4
R 4
L 3
U 1
L 1
D 4
L 3
D 1
U 2
R 4
U 1
L 4
D 3
U 2
R 4
D 3
U 3
L 4
U 2
R 1
U 1
D 3
R 3
D 1
U 1
L 3
R 1
D 2
R 4
U 2
D 4
U 4
L 4
R 4
D 2
R 3
D 3
R 4
U 1
D 3
R 4
L 1
U 1
L 1
D 4
R 1
U 4
L 3
R 2
U 4
R 3
U 1
L 3
D 2
L 2
R 3
D 1
L 4
D 2
R 3
L 4
R 4
L 2
U 1
D 1
R 3
U 4
D 4
U 4
D 4
R 4
U 4
D 1
U 1
D 4
U 1
D 4
U 4
R 4
D 4
L 4
D 1
U 5
R 5
U 5
L 1
R 4
U 5
L 2
U 4
D 1
R 1
L 4
D 2
L 5
D 5
R 3
L 2
R 1
U 1
R 5
D 2
U 1
L 1
D 4
U 4
L 2
U 4
L 1
D 4
L 1
U 4
L 3
U 3
D 2
L 3
D 2
L 3
R 1
L 2
U 1
L 2
R 2
D 5
U 4
R 1
D 5
U 2
R 3
U 4
L 4
R 5
L 2
R 5
U 5
D 3
R 2
D 1
R 2
U 4
D 1
R 4
D 5
R 4
L 4
U 5
D 1
L 2
R 1
U 3
L 5
R 5
L 2
D 3
L 2
R 3
U 2
D 2
U 3
L 5
U 3
D 1
R 1
D 3
L 2
U 1
D 2
U 4
L 4
U 5
R 5
U 5
L 5
R 3
L 1
D 1
L 3
D 4
R 1
D 1
R 3
L 4
U 4
L 1
R 5
D 4
U 2
D 6
U 6
L 6
U 2
L 2
D 6
U 2
R 2
U 1
R 6
D 6
R 5
L 4
R 3
D 4
L 6
U 1
D 2
L 5
U 2
D 1
U 6
R 4
L 4
U 3
L 3
U 6
L 4
R 4
L 5
U 2
L 1
U 1
D 5
U 4
D 5
L 5
U 3
R 6
U 5
D 6
R 3
D 1
L 3
R 5
U 5
R 4
D 3
U 1
D 1
R 1
D 1
L 6
U 2
D 4
U 6
D 4
L 6
D 4
R 6
U 4
R 2
D 2
U 1
L 2
U 5
D 4
L 2
D 1
L 3
R 1
L 1
R 6
L 3
U 5
D 3
R 5
U 1
D 5
R 4
D 2
U 6
R 1
U 3
D 2
R 1
U 6
R 2
U 5
R 4
D 6
U 1
R 2
D 6
U 5
R 1
D 6
U 2
L 3
U 3
D 1
R 4
D 1
L 3
R 4
U 2
R 6
L 3
R 6
U 6
D 1
R 4
D 2
U 6
D 3
R 2
L 1
D 5
L 6
R 4
D 3
R 2
U 2
R 4
D 1
U 2
L 6
D 3
L 3
D 6
L 4
U 6
D 2
R 3
D 7
L 7
R 4
D 2
L 1
R 7
L 7
U 7
D 3
R 2
L 4
D 1
L 2
U 6
D 6
R 5
L 5
U 3
L 5
R 2
L 5
U 3
D 6
L 1
D 3
L 7
U 1
R 7
D 1
R 2
U 5
L 3
D 2
L 2
R 2
U 2
R 1
L 5
U 2
D 2
U 1
R 4
D 4
L 6
D 3
U 4
L 3
D 3
U 2
L 4
D 2
R 3
L 6
R 3
D 6
U 1
R 3
U 7
D 5
L 1
R 7
L 5
R 1
D 7
U 4
L 3
R 4
D 1
L 5
D 3
R 6
U 5
L 5
R 5
D 5
U 7
R 1
D 1
U 5
D 3
R 2
U 6
D 5
U 3
L 3
R 4
L 8
U 2
R 1
U 4
R 5
L 6
U 4
R 3
D 2
U 7
R 7
D 1
U 1
L 3
R 4
U 6
L 8
D 4
R 5
D 6
L 1
R 8
U 2
L 7
U 5
R 4
U 1
D 1
R 8
L 6
D 6
R 1
D 1
U 2
R 1
L 6
R 5
U 1
D 1
L 2
U 7
R 4
D 6
L 3
D 7
U 1
R 7
D 1
R 6
L 2
D 4
L 3
R 7
L 7
U 8
R 7
L 5
D 8
R 1
D 4
L 3
R 7
L 4
U 7
L 3
D 7
L 4
R 8
L 5
D 2
R 3
U 6
D 1
L 7
D 1
U 2
D 8
U 7
R 1
U 8
L 4
U 4
D 5
R 2
D 7
R 5
L 4
D 4
U 4
D 5
U 2
D 5
R 8
D 7
U 2
D 7
R 3
U 6
L 6
D 5
L 5
R 3
U 5
R 5
L 3
D 3
R 3
D 5
R 6
D 3
L 1
U 6
R 4
D 4
R 8
D 6
L 8
D 2
U 1
L 9
R 8
D 6
L 8
R 6
L 8
U 4
L 6
R 5
U 1
L 5
R 8
D 3
L 8
R 9
U 9
L 5
D 5
R 2
D 1
U 4
R 1
U 2
R 2
L 8
D 1
L 3
D 2
U 1
L 8
R 5
L 2
D 9
U 4
L 5
D 3
U 4
D 8
L 4
U 1
L 1
D 2
U 6
D 8
U 3
D 2
U 3
R 5
U 6
L 1
U 1
L 8
D 9
L 4
U 4
R 3
D 6
U 5
L 8
D 4
U 4
L 8
R 6
L 4
U 5
R 5
U 4
D 4
U 4
R 7
L 8
U 6
L 9
R 8
U 5
L 2
U 5
D 3
L 7
D 4
R 1
U 3
R 6
L 9
R 8
U 8
L 7
R 1
L 1
U 3
D 8
R 3
D 1
L 5
D 2
R 4
U 9
L 1
R 1
L 5
R 4
U 8
D 6
U 6
R 9
L 3
R 9
U 1
R 9
D 5
L 4
R 6
L 3
R 2
U 9
D 8
R 4
L 6
R 5
L 6
U 6
R 8
U 10
L 2
R 1
D 8
L 9
U 4
D 1
L 9
R 6
U 7
D 7
L 6
R 6
U 2
L 3
U 6
R 4
L 10
R 1
L 5
U 2
R 6
D 2
R 9
L 4
U 4
D 1
L 7
R 8
L 1
D 9
L 6
U 6
L 10
U 1
L 2
R 2
U 3
L 5
D 4
L 8
D 7
L 5
R 3
D 7
U 10
D 10
R 8
D 3
R 7
D 5
R 10
D 9
R 6
L 3
R 1
U 8
L 3
R 9
U 4
D 3
R 8
D 10
U 5
D 8
L 3
D 8
L 3
D 6
R 6
D 1
R 3
D 10
R 4
L 2
R 10
D 4
U 1
L 9
R 9
D 10
U 4
D 2
L 5
R 8
L 2
R 4
U 5
D 1
L 3
U 4
D 7
L 11
R 2
L 1
D 3
R 8
D 5
U 6
L 11
D 7
U 3
R 4
L 6
U 2
D 1
R 5
U 9
L 3
U 4
R 9
D 11
R 1
U 8
L 7
R 6
D 11
U 8
R 3
D 8
U 6
R 8
U 8
D 9
U 6
R 7
U 11
L 10
U 11
L 10
U 10
R 11
U 10
L 10
R 6
L 11
D 1
L 5
D 4
R 1
L 4
U 2
D 8
L 10
R 8
D 10
L 2
D 5
L 11
R 6
L 9
U 3
L 1
R 11
L 8
D 5
U 11
D 10
R 5
U 9
L 4
U 5
R 8
L 8
U 1
L 11
U 6
R 6
D 5
R 5
L 4
U 2
L 6
D 3
L 9
R 6
U 3
D 11
R 1
D 11
U 4
R 9
D 3
L 7
D 7
L 11
D 5
L 5
D 8
R 10
U 2
L 6
U 10
D 10
R 4
D 6
U 10
R 3
U 5
R 4
U 3
D 11
R 7
D 10
L 6
U 2
L 11
D 12
L 8
U 11
D 10
L 2
D 10
L 2
R 3
D 6
L 9
D 9
U 2
L 5
R 3
U 3
D 1
U 7
L 5
R 2
U 7
D 12
U 5
L 1
R 3
D 8
U 3
D 12
R 5
L 11
U 5
L 2
R 3
L 7
R 9
U 5
D 3
L 4
D 8
U 6
R 11
D 10
U 3
R 4
L 3
R 9
U 7
D 5
L 10
R 1
U 8
D 9
L 12
R 6
D 11
U 7
D 11
R 2
D 9
R 11
U 12
R 2
D 11
R 5
L 1
U 6
D 4
L 7
U 10
L 5
D 8
R 7
L 7
R 6
U 7
L 9
U 12
L 9
U 2
R 6
U 11
L 2
D 3
R 3
U 3
R 8
D 5
R 10
D 12
R 1
D 3
U 8
D 2
L 4
U 7
R 1
L 11
R 7
U 11
D 3
R 9
L 1
D 1
R 11
L 12
R 1
U 11
R 9
L 12
R 5
D 11
R 10
U 3
L 13
U 11
L 11
R 3
L 1
U 12
L 7
U 2
D 7
U 9
R 2
U 7
L 10
R 9
U 13
D 8
R 7
D 7
L 1
U 2
L 9
D 9
L 13
R 13
L 12
U 3
L 1
R 5
D 8
U 2
D 12
R 9
D 11
R 12
L 10
U 9
R 1
U 11
L 5
R 10
L 7
R 9
U 11
D 3
R 13
L 11
U 4
R 1
U 7
L 2
R 3
L 6
D 4
R 8
L 8
U 10
D 12
U 9
R 7
D 1
R 8
U 7
R 1
D 8
L 6
D 13
L 6
R 4
U 13
L 2
R 5
D 2
U 1
L 6
R 7
D 11
L 9
R 11
U 3
L 8
R 6
L 10
U 9
R 1
U 3
R 1
L 6
R 4
L 3
D 8
U 3
D 3
U 8
L 12
R 5
L 3
D 8
L 7
D 3
R 5
L 3
R 11
L 4
U 6
R 7
L 11
D 4
R 5
U 6
R 13
D 11
U 2
D 12
L 8
R 3
D 4
L 12
D 7
R 14
U 8
R 12
U 14
R 1
L 3
R 13
U 14
D 12
R 4
U 4
R 14
D 10
U 11
R 10
D 13
U 8
R 13
L 9
U 6
D 7
U 9
L 11
U 10
L 14
U 7
L 5
D 1
U 8
R 14
U 12
L 1
R 4
L 7
D 2
R 5
D 3
R 14
U 6
D 7
R 2
L 8
U 14
D 12
U 12
L 5
D 5
L 1
U 3
L 10
R 4
U 2
R 11
D 6
U 12
D 5
U 11
D 5
L 8
D 1
L 7
D 1
U 9
D 6
L 10
D 7
L 10
R 7
D 11
L 9
U 13
D 10
U 10
R 13
L 2
R 13
U 3
R 8
D 7
L 2
U 1
L 7
D 11
U 6
D 12
L 3
R 3
D 1
R 2
D 2
U 4
D 1
L 12
U 5
L 3
R 13
D 9
L 3
D 10
L 5
R 13
D 9
L 7
R 12
L 8
U 1
R 5
L 3
D 7
R 4
U 6
R 4
U 15
D 13
R 3
L 14
R 1
U 3
R 4
U 12
R 7
D 7
U 1
R 12
D 9
R 14
U 10
R 1
U 6
D 10
R 14
D 12
R 6
D 13
U 14
R 15
U 9
L 6
R 7
L 13
R 4
D 8
R 2
L 6
U 13
L 15
D 5
U 4
R 14
D 5
L 6
U 3
R 10
L 12
U 9
D 5
L 4
U 1
R 10
L 3
U 5
R 12
D 14
L 10
R 9
D 2
U 6
D 6
U 1
L 8
R 3
D 8
R 1
D 5
L 10
D 11
L 5
R 6
D 8
R 2
D 12
L 7
R 14
D 8
L 8
U 14
R 13
L 7
D 8
U 3
L 6
U 6
D 4
R 9
L 11
R 2
U 10
R 14
D 8
R 2
U 10
R 7
U 3
R 4
D 9
U 9
D 11
U 5
D 9
R 14
U 13
D 5
L 7
R 2
D 9
R 2
U 9
D 8
L 12
R 12
L 3
R 6
U 16
R 15
D 8
U 3
R 1
L 8
D 11
L 6
R 5
L 14
D 15
U 1
R 10
D 11
R 16
D 1
U 6
L 12
U 3
D 9
U 1
D 13
L 14
R 8
L 9
D 3
U 16
R 3
L 11
R 13
L 10
U 10
L 3
R 7
L 6
R 2
D 4
U 15
R 13
U 10
D 1
L 3
D 15
L 9
R 8
D 14
R 11
L 11
D 4
U 15
L 2
U 4
D 3
U 6
D 5
U 15
L 8
D 14
R 1
U 8
L 7
D 8
R 8
U 1
D 8
R 11
D 4
U 14
R 11
D 16
U 4
D 9
R 11
L 16
D 5
L 9
R 6
U 9
D 16
R 4
L 11
U 11
D 6
U 13
D 14
U 13
L 8
U 9
D 3
R 4
L 8
D 7
L 12
D 6
R 14
L 11
U 7
D 4
U 1
L 11
U 15
L 5
D 8
U 2
R 15
L 5
R 7
L 8
R 15
U 8
D 7
U 17
D 5
U 15
R 8
U 15
L 7
U 5
D 11
L 4
U 11
D 13
R 10
D 12
L 16
D 9
L 17
U 1
R 10
D 1
R 16
U 6
L 2
D 7
U 8
D 12
L 15
U 16
R 5
L 13
D 2
U 7
L 14
D 6
L 8
R 12
U 4
D 7
U 4
R 3
U 8
D 5
L 4
U 3
R 13
L 14
R 7
U 11
D 9
R 1
D 15
R 11
L 8
R 7
D 17
U 13
R 15
D 5
R 15
U 7
R 10
U 14
D 2
L 7
R 13
D 10
U 16
D 6
R 10
D 4
U 1
R 14
U 2
L 2
U 1
L 13
D 2
U 16
D 1
U 8
L 7
U 17
D 9
L 10
U 16
D 13
U 15
D 12
L 6
D 9
L 3
R 17
U 16
L 6
D 11
R 11
L 11
U 2
R 15
D 11
R 10
U 2
D 18
U 18
D 18
L 4
U 1
R 1
L 7
U 7
L 12
U 10
R 13
D 16
R 1
D 11
L 12
R 7
D 13
L 10
D 6
U 1
L 14
R 13
D 14
L 3
D 18
L 2
U 9
L 7
R 18
L 11
U 16
R 9
U 8
D 5
L 15
R 16
D 17
U 16
L 10
U 4
R 1
L 10
U 4
L 8
U 5
L 5
R 18
U 11
R 18
U 12
D 18
R 9
L 2
D 10
L 15
D 2
R 7
D 16
L 4
R 13
L 4
R 2
L 14
D 4
R 15
D 14
R 13
L 9
D 1
U 7
R 17
L 6
U 1
L 6
D 6
U 5
L 5
R 10
U 1
R 9
U 3
D 2
L 4
U 1
D 12
R 5
U 3
R 15
L 18
R 3
D 1
L 6
D 16
U 15
D 6
R 8
D 4
R 14
U 10
R 10
D 5
L 14
U 6
L 15
R 10
D 17
R 7
L 13
R 12
D 4
R 11
D 2
L 1
R 13
U 6
D 16
R 5
D 17
U 17
D 11
R 15
L 4
D 13
L 11
U 16
L 3
D 5
U 1
R 16
L 10
U 7
D 9
R 10
L 4
D 4
L 7
D 12
U 5
D 10
R 2
L 9
D 6
U 14
R 4
U 2
L 19
R 15
L 13
R 12
L 8
D 15
U 13
L 4
U 13
D 3
U 7
L 10
R 8
D 14
R 3
L 10
D 3
U 5
L 15
R 5
L 2
R 10
D 5
R 13
L 18
R 7
L 6
U 12
D 16
R 3
L 11
D 7
R 1
U 9
R 9
D 6
L 1
D 13
U 10
L 17
R 10
L 12
R 8
D 10
R 12
U 12
D 14
U 1
D 4
U 2
L 9
R 3
U 2
R 19
D 1
L 4
R 6
L 4
D 4
R 16
D 17
R 3
D 9
L 8
R 15
D 4
U 13
D 4
U 17
D 15
L 17
D 1
R 14
L 18
U 2
R 11
U 5'''